In [ ]:
def _post_with_retry(
    self,
    path: str,
    headers: Dict[str, str],
    payload: Optional[Dict[str, Any]] = None,
    files: Optional[Dict[str, Any]] = None,
    retries: Optional[int] = 5,
    timeout: Optional[int] = 300,
) -> Optional[Dict[str, Any]]:
    """
    Perform an HTTP POST request with retry logic.
    
    Args:
        path: API endpoint path
        headers: HTTP headers
        payload: Request payload
        files: Optional files to upload
        retries: Maximum number of retries
        timeout: Request timeout in seconds
        
    Returns:
        Processed response data or None if all retries failed
    """
    url = self.base_url + path
    remaining_retries = retries
    
    while remaining_retries > 0:
        try:
            # Handle payload differently if files are provided to avoid conflicts
            if files is not None:
                response = requests.post(
                    url,
                    headers=headers,
                    data=payload,  # Use data instead of json when files are present
                    files=files,
                    verify=self.verify_ssl,
                    timeout=timeout,
                )
            else:
                response = requests.post(
                    url,
                    headers=headers,
                    json=payload,  # Use json for proper serialization
                    verify=self.verify_ssl,
                    timeout=timeout,
                )
            
            # Check for HTTP errors
            if response.status_code >= 300:
                detail = f"Error code from API: {response.status_code} {response.text}"
                logger.error(f"Request failed: {detail}")
                
                # For 5xx errors, retry. For 4xx, process and return the response
                if response.status_code < 500:
                    # For 4xx errors, read the content and return it
                    try:
                        return response.json()
                    except Exception:
                        return {"text": response.text, "status": response.status_code}
            else:
                # Success - fully process the response
                try:
                    # Try to get JSON first
                    content = response.json()
                    logger.debug(f"Successfully parsed response as JSON")
                    return content
                except json.JSONDecodeError as json_error:
                    # If JSON fails, get text instead
                    logger.warning(f"JSON parsing failed: {json_error}, returning text instead")
                    text = response.text
                    logger.debug(f"Successfully read response text (length: {len(text)})")
                    try:
                        # Try to parse the text as JSON manually
                        return json.loads(text)
                    except json.JSONDecodeError:
                        # Return text if it's not JSON
                        return {"text": text, "status": response.status_code}
                
        except Exception as e:
            logger.error(f"Exception during request to {url}: {str(e)}")
        
        # If we get here, we need to retry
        remaining_retries -= 1
        if remaining_retries > 0:
            logger.info(
                f"Retrying {url} in {self.retry_delay} seconds. "
                f"Attempts remaining: {remaining_retries}"
            )
            time.sleep(self.retry_delay)
    
    # All retries failed
    logger.error(f"Failed to POST to {url} after {retries} attempts")
    return None